### Importing necessary libraries

In [1]:
import sagemaker
import boto3
from sagemaker.amazon.amazon_estimator import get_image_uri 
from sagemaker.session import s3_input, Session

In [13]:
bucket_name = 'himabankapplication'

# Checking the region to create s3 bucket
my_region = boto3.session.Session().region_name
print(my_region)

us-east-1


In [15]:
s3 = boto3.resource('s3')
try:
    if my_region == 'us-east-1':
        s3.create_bucket(Bucket=bucket_name)
        print("S3 bucket created successfully")
except Exception as e:
    print("S3 error: ", e)

S3 bucket created successfully


In [18]:
# Setting an output path where trained model will be saved
prefix = 'xgboost-builtin-algm'
output_path = 's3://{}/{}/output'.format(bucket_name, prefix)
print(output_path)

s3://himabankapplication/xgboost-builtin-algm/output


### Downloading dataset and storing in S3 bucket

In [19]:
import pandas as pd
import urllib

try:
    urllib.request.urlretrieve ("https://d1.awsstatic.com/tmt/build-train-deploy-machine-learning-model-sagemaker/bank_clean.27f01fbbdf43271788427f3682996ae29ceca05d.csv", "bank_clean.csv")
    print('Success: downloaded bank_clean.csv.')
except Exception as e:
    print('Data load error: ',e)
    
try:
    model_data = pd.read_csv('./bank_clean.csv',index_col=0)
    print('Success: Data loaded into dataframe.')
except Exception as e:
    print('Data load error: ',e)

Success: downloaded bank_clean.csv.
Success: Data loaded into dataframe.


In [20]:
# Splitting data into train and test

import numpy as np

train_data, test_data = np.split(model_data.sample(frac=1, random_state=1729), [int(0.7 * len(model_data))])
print(train_data.shape, test_data.shape)

(28831, 61) (12357, 61)


In [24]:
# Saving train data into buckets

import os

# In AWS Sagemaker, dependent feature should be your irst column. Here, y_yes and y_no are dependent features. 
# Select any one of those.

pd.concat([train_data['y_yes'], train_data.drop(['y_no', 'y_yes'], 
                                                axis=1)], 
                                                axis=1).to_csv('train.csv', index=False, header=False)
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')
s3_input_train = sagemaker.TrainingInput(s3_data='s3://{}/{}/train'.format(bucket_name, prefix), content_type='csv')

In [22]:
# Saving train data into buckets
pd.concat([test_data['y_yes'], test_data.drop(['y_no', 'y_yes'], axis=1)], axis=1).to_csv('test.csv', index=False, header=False)
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'test/test.csv')).upload_file('test.csv')
s3_input_test = sagemaker.TrainingInput(s3_data='s3://{}/{}/test'.format(bucket_name, prefix), content_type='csv')

### Building models using xgboost algorithm in AWS

In [30]:
from sagemaker import image_uris

# this line automatically looks for the XGBoost image URI and builds an XGBoost container.
# specify the repo_version depending on your preference.
container = image_uris.retrieve('xgboost', 
                                boto3.Session().region_name,
                                '1.0-1')

In [31]:
# initialize hyperparameters
hyperparameters = {
        "max_depth":"5",
        "eta":"0.2",
        "gamma":"4",
        "min_child_weight":"6",
        "subsample":"0.7",
        "objective":"binary:logistic",
        "num_round":50
        }

In [35]:
# construct a SageMaker estimator that calls the xgboost-container
estimator = sagemaker.estimator.Estimator(image_uri=container, 
                                          hyperparameters=hyperparameters,
                                          role=sagemaker.get_execution_role(),
                                          instance_count=1, 
                                          instance_type='ml.m5.2xlarge', 
                                          volume_size=5, # 5 GB 
                                          output_path=output_path,
                                          use_spot_instances=True,
                                          max_run=300,
                                          max_wait=600)

In [36]:
estimator.fit({'train': s3_input_train,'validation': s3_input_test})

2022-07-01 13:48:37 Starting - Starting the training job...
2022-07-01 13:49:03 Starting - Preparing the instances for trainingProfilerReport-1656683317: InProgress
......
2022-07-01 13:50:04 Downloading - Downloading input data...
2022-07-01 13:50:23 Training - Downloading the training image...
2022-07-01 13:51:03 Training - Training image download completed. Training in progress..INFO:sagemaker-containers:Imported framework sagemaker_xgboost_container.training
INFO:sagemaker-containers:Failed to parse hyperparameter objective value binary:logistic to Json.
Returning the value itself
INFO:sagemaker-containers:No GPUs detected (normal if no gpus installed)
INFO:sagemaker_xgboost_container.training:Running XGBoost Sagemaker in algorithm mode
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
[13:51:03] 28831x59 matrix with 1701029 entries loaded from /opt/ml/input/data/train?format=csv&

### Deploying model as endpoint

In [39]:
xgb_predictor = estimator.deploy(initial_instance_count=1,instance_type='ml.t2.xlarge')

-------!

### Prediction of the Test Data

In [43]:
from sagemaker.serializers import CSVSerializer
test_data_array = test_data.drop(['y_no', 'y_yes'], axis=1).values #load the data into an array
# xgb_predictor.content_type = 'text/csv' # set the data type for an inference
xgb_predictor.serializer = CSVSerializer() # set the serializer type
predictions = xgb_predictor.predict(test_data_array).decode('utf-8') # predict!
predictions_array = np.fromstring(predictions[1:], sep=',') # and turn the prediction into an array
print(predictions_array.shape)

(12357,)


In [44]:
predictions_array

array([0.05214286, 0.05660191, 0.05096195, ..., 0.03436061, 0.02942475,
       0.03715819])

In [45]:
cm = pd.crosstab(index=test_data['y_yes'], columns=np.round(predictions_array), rownames=['Observed'], colnames=['Predicted'])
tn = cm.iloc[0,0]; fn = cm.iloc[1,0]; tp = cm.iloc[1,1]; fp = cm.iloc[0,1]; p = (tp+tn)/(tp+tn+fp+fn)*100
print("\n{0:<20}{1:<4.1f}%\n".format("Overall Classification Rate: ", p))
print("{0:<15}{1:<15}{2:>8}".format("Predicted", "No Purchase", "Purchase"))
print("Observed")
print("{0:<15}{1:<2.0f}% ({2:<}){3:>6.0f}% ({4:<})".format("No Purchase", tn/(tn+fn)*100,tn, fp/(tp+fp)*100, fp))
print("{0:<16}{1:<1.0f}% ({2:<}){3:>7.0f}% ({4:<}) \n".format("Purchase", fn/(tn+fn)*100,fn, tp/(tp+fp)*100, tp))


Overall Classification Rate: 89.7%

Predicted      No Purchase    Purchase
Observed
No Purchase    91% (10785)    34% (151)
Purchase        9% (1124)     66% (297) 



### Deleting The Endpoints

In [ ]:
# Specify the name of your endpoint
endpoint_name='sagemaker-xgboost-2022-07-01-13-52-51-694'

# Create a low-level SageMaker service client.
sagemaker_client = boto3.client('sagemaker', region_name=my_region)

# Delete endpoint
sagemaker_client.delete_endpoint(EndpointName=endpoint_name)


In [55]:
## Deleting endpoint configurations as well

endpoint_cfg_1 = 'sagemaker-xgboost-2022-07-01-14-15-59-246'
endpoint_cfg_2 = 'sagemaker-xgboost-2022-07-01-13-52-51-694'

# Delete endpoint configuration
sagemaker_client.delete_endpoint_config(EndpointConfigName=endpoint_cfg_1)
sagemaker_client.delete_endpoint_config(EndpointConfigName=endpoint_cfg_2)

{'ResponseMetadata': {'RequestId': '88864d6e-11e7-481c-b77c-ed0a64beacc7',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '88864d6e-11e7-481c-b77c-ed0a64beacc7',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Fri, 01 Jul 2022 14:40:33 GMT'},
  'RetryAttempts': 3}}

In [ ]:
bucket_to_delete = boto3.resource('s3').Bucket(bucket_name)
bucket_to_delete.objects.all().delete()